In [1]:
import json, yfinance as yf

In [2]:
# from tqdm.notebook import tqdm

In [3]:
import numpy as np
import pandas as pd

In [4]:
import import_ipynb
from feeds import LiveFeed,BackFeed,DataFeed,clean_feed,clean_feed_tickers

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb


In [5]:
from featfuncs import add_addl_features_feed,add_sym_feature_feed

In [6]:
import pickle
# from matplotlib import pyplot as plt

In [39]:
def compute_gaps(feed,topk=5):
    dates=[]
    for t in feed.ndata:
        for d in feed.ndata[t]:
            dates+=[d]
    dates=list(set(dates))
    gaps={}
    stocks={}
    for d in dates:
        gaps[d]={}
        for t in feed.ndata:
            if d in feed.ndata[t]:
                df=feed.ndata[t][d]
                dts=df.Date.unique()
                if len(dts)>1:
                    endcl=df.loc[df['Date']==dts[0]]['Close_n'].values[0]
                    startcl=df.loc[df['Date']==dts[1]]['Open_n'].values[0]
                    gaps[d][t]=abs(startcl-endcl)
        g=gaps[d]
        stocks[d]=pd.DataFrame([(t,g[t]) for t in g],columns=['ticker','gap']).sort_values('gap',ascending=False)
        stocks[d]=stocks[d].iloc[0:topk]
    return stocks

In [8]:
def market_cap(t):
    try:
        bs=yf.Ticker(t).balance_sheet
        c=bs.loc[bs.index=='Common Stock'].values[-1][-1]
        p=yf.Ticker(t).history(period='1d',interval='1d').iloc[0]['Close']
        mc=(p*c/80)/1000000
    except: mc=0
    return mc

In [9]:
market_capD={}
def compute_market_caps(M,N):
    global market_capD,nsetickers
    for t in tqdm(list(nsetickers.keys())[M:N]):
        market_capD[t+'.NS']=market_cap(t+'.NS')
    caps=pd.DataFrame([(t,market_capD[t]) for t in market_capD],
                      columns=['ticker','cap']).sort_values('cap',ascending=False)
    return caps

In [10]:
def vol_est(t):
    try:
        df=yf.Ticker(t).history(period="1y",interval="1d")
        vol=np.mean((100*abs(df['Close']-df['Open'])/df['Close']).values)
        return vol
    except:
        return 0

In [11]:
volsD={}
def compute_vols(M,N):
    global volsD,nsetickers
    volsD={}
    for t in tqdm(list(nsetickers.keys())[M:N]):
        volsD[t+'.NS']=vol_est(t+'.NS')
    volsf=pd.DataFrame([(t,volsD[t]) for t in volsD],
                      columns=['ticker','vol']).sort_values('vol',ascending=False)
    return volsf

Debugging

Market caps of all tickers - to get subset of top K 